# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [2]:
%run ../config/bootstrap.py

In [3]:
import pandas as pd
from utils import get_project_root, save_parquet_in_chunks , load_partitioned_parquet
import basedosdados as bd

In [7]:
project_root = get_project_root() 
project_root
#billing_id = 'ufrgs-ppgc-dengue'

PosixPath('/Users/silmara.barnabe/Desktop/Projects/arbovirus')

In [ ]:
/Users/silmara.barnabe/Desktop/Projects/arbovirus/data/4_gold/dim_cnes_w_notifications/dim_cnes_w_notifications.parquet_part_1.parquet

In [10]:
path = project_root / "data/4_gold/dim_cnes_w_notifications/dim_cnes_w_notifications.parquet_part_1.parquet"
df = pd.read_parquet(path)
df.head()

,ano,id_cnes,target,obito,flag_caso_encerrado,qt,id_unidade_cnes,nu_cnpj_mantenedora,id_pfpj,nivel_dep,no_razao_social,no_fantasia,no_logradouro,nu_endereco,no_complemento,no_bairro,id_cep,id_regiao_saude,id_micro_regiao,id_distrito_sanitario,id_distrito_administrativo,nu_telefone,nu_fax,no_email,nu_cpf,nu_cnpj,id_atividade,id_clientela,nu_alvara,data_expedicao,id_orgao_expedidor,data_val_lic_sani,id_lic_sani,id_unidade,id_turno_atendimento,id_estado_gestor,id_municipio_gestor,data_atualizacao,id_usuario,id_cpfdiretorcln,reg_diretorcln,st_adesao_filantrop,id_motivo_desab,no_url,nu_latitude,nu_longitude,data_atualizacao_geolocalizacao,no_usuario_geo,id_natureza_jur,id_estab_sempre_aberto,st_geracredito_gerente_sgif,st_conexao_internet,id_tipo_unidade,no_fantasia_abrev,id_gestao,data_atualizacao_origem,id_tipo_estabelecimento,id_atividade_principal,st_contrato_formalizado,id_tipo_abrangencia,st_coworking,hash,data_inicio_vigencia_silver,data_fim_vigencia_silver,data_atualizacao_silver,vigencia_silver
0,2024,2103311,0,0,338,338,3159502103311,18413187000110,3,3,PREFEITURA MUNICIPAL DE SANTA RITA DO ITUETO,CENTRO DE SAUDE DE SANTA RITA DO ITUETO,RUA MARIA INES,175,None,CENTRO,35225000,0016,None,None,None,(33)32651184,None,secretaria.sri@gmail.com,None,None,04,01,None,None,None,None,None,02,04,31,315950,2025-04-15 00:00:00,SAUDE,04850919685,159687,None,None,None,"-19,35961","-41,37983",2023-11-08 00:00:00,APARECIDA,1244,N,None,S,None,None,M,2001-10-30 00:00:00,001,012,None,03,N,7fef0595a0b56a04a9b69d64a28be60b5143d850bf11187833b131334c0bc0eb,2025-08-17,9999-12-31,2025-08-17,1.0
1,2024,2119420,0,0,2562,2562,3108602119420,18017442000106,3,3,MUNICIPIO DE BRASILIA DE MINAS,HOSPITAL MUNICIPAL SENHORA SANTANA,AVENIDA BIAS FORTES,S/N,CENTRO,DONA JOAQUINA,39330000,0030,None,None,None,(38)32311803,None,hmss@brasiliademinas.mg.gov.br,None,None,04,03,0081/03,2003-07-17 00:00:00,None,None,None,05,06,31,310860,2025-05-08 00:00:00,SMSBM,45734399604,None,None,None,None,"-16,219413481866532","-44,42647933959961",2019-05-08 00:00:00,SMSBM,1244,S,None,S,None,None,M,2002-03-12 00:00:00,006,009,None,05,N,c8c883400a30f53a019ecae520aa979455ff55e4afb3e0bfdd2159869d28688c,2025-08-17,9999-12-31,2025-08-17,1.0
2,2024,2144654,0,0,384,384,3122602144654,None,3,1,IRMANDADE NOSSA SENHORA DAS GRACAS DOM JOAQUIM,HOSPITAL NOSSA SENHORA DAS GRACAS DOM JOAQUIM,RUA DR ARY ASCENCAO D OLIVEIRA,85,None,CENTRO,35865000,0009,None,None,None,(31)38661131,None,None,None,22056741000127,04,01,01,2020-12-08 00:00:00,2,2020-12-31 00:00:00,1,05,06,31,312260,2025-04-29 00:00:00,SAUDE,01192593626,74739,2,None,None,"-18,959823","-43,251191",2019-07-12 00:00:00,ELIANE,3999,S,None,S,None,None,M,2002-03-21 00:00:00,006,009,S,03,N,312f8c3c6a5ed01cca4e3ff8a7f2e0f8c90993a493efc99a6e6af9f2af8d9fbf,2025-08-17,9999-12-31,2025-08-17,1.0
3,2024,2307898,0,0,76,76,2102302307898,06052138000110,3,3,PREFEITURA MUNICPAL DE BURITI BRAVO,HOSPITAL MUNICIPAL ZUZA COELHO,RUA JOAQUIM LEITE,S/N,None,LARANJEIRAS,65685000,13,None,None,None,(09)85721010,None,None,None,None,04,01,None,None,None,None,None,05,06,21,210230,2025-06-26 00:00:00,CNESBB,03764609389,None,None,None,None,"-5,829987","-43,834312",2019-09-02 00:00:00,CNESBB,1244,S,None,S,None,None,M,2001-10-30 00:00:00,006,001,None,None,N,8105bef67dee2f643327b0bc728d4a8bb85d46fc31842eca466bb0031ca05dd1,2025-08-17,9999-12-31,2025-08-17,1.0
4,2024,2461625,0,0,25,25,2111742461625,01566688000134,3,3,PREFEITURA MUNICIPAL DE ALEXANDRE COSTA,HOSPITAL MUNICIPAL MARIA JOSE MACHADO,RUA CONEGO ADERSON,S/N,None,CENTRO,65783000,14,None,None,None,(99) 5691073,None,None,None,None,04,01,None,None,None,None,None,05,06,21,211174,2022-08-15 00:00:00,MESTRE,10458875368,None,None,10,None,"-5,258764391608838","-44,054274559020996",2020-02-20 00:00:00,MESTRE,1244,S,None,S,None,None,M,2003-11-09 00:00:00,006,009,None,None,None,e4d79c86fb2fdee65f018b1e7c3b7f3b126535b19279e4c2ded0a50b4d020911,2025-08-17,9999-12-31,2025-08-17,1.0


### Porto Alegre

In [26]:
df_poa = df[df['id_municipio_gestor'] == '431490']
df_poa.shape

(880, 66)

## Rio de Janeiro

In [13]:
df_rj = df[df['id_municipio_gestor'] == '330455']
df_rj.shape

(1856, 66)

In [25]:
df_rj.no_bairro.unique()

array(['MANGUINHOS', 'CAMPO GRANDE', 'GUARATIBA', 'COPACABANA',
       'BOTAFOGO', 'SANTA CRUZ', 'PENHA', 'PEDRA DE GUARATIBA',
       'CAMPINHO', 'ILHA DO FUNDAO', 'PILARES', 'ROCHA MIRANDA',
       'SENADOR CAMARA', 'CORDOVIL', 'RIO COMPRIDO', 'CENTRO', 'INHOAIBA',
       'JARDIM BANGU', 'BANCARIOS', 'LINS DE VASCONCELOS', 'ZUMBI',
       'MEIER', 'JARDIM AMERICA', 'RAMOS', 'GAVEA', 'SAO CRISTOVAO',
       'REALENGO', 'SEPETIBA', 'FREGUESIA JACAREPAG', 'LARANJEIRAS',
       'IRAJA', 'MARE', 'JACAREPAGUA', 'SAO CONRADO', 'PACIENCIA',
       'CIDADES DE DEUS', 'MADUREIRA', 'DEL CASTILHO',
       'QUINTINO BOCAIUVA', 'LEBLON', 'SENADOR VASCONCELOS',
       'HONORIO GURGEL', 'ENGENHEIRO LEAL', 'ILHA DO GOVERNADOR',
       'PADRE MIGUEL', 'BANGU', 'TIJUCA', 'ALTO DA BOA VISTA',
       'VILA ISABEL', 'ACARI', 'BARRA DA TIJUCA', 'JACAREZINHO',
       'VILA KOSMOS', 'PRACA DA BANDEIRA', 'GRAJAU', 'COELHO NETO',
       'VILA MILITAR', 'SANTISSIMO', 'VILA KENNEDY', 'CATUMBI',
       'ENGENHO N

In [18]:
df_resumo = pd.crosstab(
    df_rj["no_bairro"],
    df_rj["target"],
    values=df_rj["qt"],
    aggfunc="sum",
    margins=True,
    margins_name="Notificações Totais",
)
df_resumo = df_resumo.reset_index()
df_resumo = df_resumo.rename(columns={"no_bairro": "Bairro"})

In [21]:
df_resumo.sort_values(by="Notificações Totais", ascending=False, inplace=True)
df_resumo.head(10)

target,Bairro,0,Notificações Totais
148,Notificações Totais,157983,157983
21,CAMPO GRANDE,15685,15685
118,SANTA CRUZ,9494,9494
9,BARRA DA TIJUCA,5944,5944
60,GUARATIBA,5581,5581
71,JACAREPAGUA,5267,5267
8,BANGU,4678,4678
12,BENFICA,3684,3684
133,TIJUCA,3590,3590
14,BOTAFOGO,3589,3589


In [24]:
df_resumo.sort_values(by="Bairro", ascending=False).head(20)

target,Bairro,0,Notificações Totais
147,ZUMBI,179,179
146,VISTA ALEGRE,108,108
145,VILA VALQUERE,1,1
144,VILA VALQUEIRE,34,34
143,VILA MILITAR,1325,1325
142,VILA KOSMOS,770,770
141,VILA KENNEDY,783,783
140,VILA ISABEL,1332,1332
139,VILA DA PENHA,472,472
138,VIGARIO GERAL,357,357


In [ ]:
df_resumo.sort_values(by="Bairro", ascending=False).head(20)